<a href="https://colab.research.google.com/github/chaitanyasaip/Python-Project/blob/main/Mistral_7B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub
#!python -m pip install --upgrade pip
!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch==2.1.0
#!pip install -q -U xformers
!pip install -q -U scipy
#!pip install -U flash-attn -q
!pip install transformers
!pip install streamlit
!pip install langchain
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 48.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
cache_dir=''

In [3]:
### DEFINE THE HUGGING SPACE MODEL

## Model A
# model_name_A = "Trelis/falcon-7b-chat-llama-style"
# model_name_A = "Trelis/Llama-2-7b-chat-hf-32k"
# model_name_A = "Yukang/Llama-2-7b-longlora-100k-ft"
# model_name_A = "Yukang/LongAlpaca-13B"
# model_name_A="meta-llama/Llama-2-13b-chat-hf"
# model_name_A = "codellama/CodeLlama-13b-Instruct-hf"
# model_name_A = "codellama/CodeLlama-34b-Instruct-hf"

# ## Model B
# model_name_B = "Trelis/Llama-2-7b-chat-hf-32k"
# model_name_B = "Yukang/Llama-2-13b-chat-longlora-32k-sft"

# ## Model C
model_name_C = "mistralai/Mistral-7B-Instruct-v0.1"

# # 1.1B model
# model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"

In [4]:
import transformers
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TextStreamer

In [5]:
# Load the model in 4-bit to allow it to fit in a free Google Colab runtime with a CPU and T4 GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, #adds speed with minimal loss of quality.
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

## Model A
#model_A = AutoModelForCausalLM.from_pretrained(
#    model_name_A,
#    quantization_config=bnb_config,
    # rope_scaling={"type": "linear", "factor": 8.0},
#    device_map='auto',
#    trust_remote_code=True,
#    use_flash_attention_2=True, # works with Llama models and reduces memory reqs
#    cache_dir=cache_dir)

# ## Model B
# model_B = AutoModelForCausalLM.from_pretrained(
#     model_name_B,
#     quantization_config=bnb_config,
#     device_map='auto',
#     trust_remote_code=True,
#     cache_dir=cache_dir)

# ## Model C
model_C = AutoModelForCausalLM.from_pretrained(
     model_name_C,
     quantization_config=bnb_config,
     device_map='auto',
     trust_remote_code=True,
     cache_dir=cache_dir)


#model = AutoModelForCausalLM.from_pretrained(
    #device_map="auto"
#    model_name, offload_folder = offload_dir, offload_state_dict = True
#)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
print(model_C.config)

MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.35.2",
  "use_cache": true,


In [7]:
# Model A
# tokenizer_A = AutoTokenizer.from_pretrained(model_name_A, cache_dir=cache_dir, use_fast=True) # will use the Rust fast tokenizer if available
### above I'm using model B for the tokenizer

# # Model B
# tokenizer_B = AutoTokenizer.from_pretrained(model_name_B, cache_dir=cache_dir, use_fast=True) # will use the Rust fast tokenizer if available

# # Model C
tokenizer_C = AutoTokenizer.from_pretrained(model_name_C, cache_dir=cache_dir, use_fast=True) # will use the Rust fast tokenizer if av

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [8]:
from IPython.display import display, HTML

# Define a stream *without* function calling capabilities
def generate(model, tokenizer, user_prompt):
    system_prompt = ''

    # # Guanaco style for TinyLlama
    # B_INST, E_INST = "### Human:", "### Assistant:"
    # B_SYS, E_SYS = "\n", "\n\n"

    # # Llama style (with no system message)
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "", ""

    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"

    inputs = tokenizer([prompt], return_tensors="pt").to('cuda')
    shape = inputs.input_ids.shape
    print(f"Length of input is {shape[1]}")
    result = model.generate(**inputs, max_new_tokens=500, pad_token_id=tokenizer.eos_token_id, do_sample=False)

    # Decode the generated text back to readable string
    result_str = tokenizer.decode(result[0], skip_special_tokens=True)

    return result_str

In [9]:
prompt = 'List the planets in our solar system. Respond only with the list of planets.'

# display(HTML(f"<b>{model_name_A}:</b><br>"))
# result = generate(model_A,tokenizer_A,prompt)
# print(result)

# display(HTML(f"<br><b>{model_name_B}:</b><br>"))
# result = generate(model_B,tokenizer_B,prompt)
# print(result)

display(HTML(f"<br><b>{model_name_C}:</b><br>"))
result = generate(model_C,tokenizer_C,prompt)
print(result)

Length of input is 27
[INST] List the planets in our solar system. Respond only with the list of planets. [/INST]

Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune.


In [10]:
prompt = 'What are the air quality index of top 10 polluted citites?'

display(HTML(f"<br><b>{model_name_C}:</b><br>"))
result = generate(model_C,tokenizer_C,prompt)
print(result)

Length of input is 27
[INST] What are the air quality index of top 10 polluted citites? [/INST]

1. Delhi, India: 290
2. Beijing, China: 250
3. Mumbai, India: 240
4. Kolkata, India: 230
5. Shanghai, China: 220
6. Dhaka, Bangladesh: 210
7. Karachi, Pakistan: 200
8. Chennai, India: 190
9. Lahore, Pakistan: 180
10. Baghdad, Iraq: 170

Note: These values are approximate and may vary depending on the source and method of measurement.


In [11]:
def interactive_chat():
    while True:
        prompt = input("Please enter your question: ")
        display(HTML(f"<br><b>{model_name_C}:</b><br>"))
        result = generate(model_C,tokenizer_C,prompt)
        print(result)

        # Ask user if they are satisfied
        feedback = input("Are you satisfied with the answer? (yes/no): ")
        if feedback.lower() == 'yes':
            break
# Start interactive chat
interactive_chat()

Please enter your question: Explain theory of relativity to a 10 year old


Length of input is 23
[INST] Explain theory of relativity to a 10 year old [/INST]

The theory of relativity is a scientific explanation of how the world works. It was created by a scientist named Albert Einstein.

The theory of relativity is actually made up of two parts: the special theory of relativity and the general theory of relativity.

The special theory of relativity is all about how time and space work together. It tells us that time and space are not separate things, but they are actually connected. This means that things that are moving fast can seem to be moving slower than things that are moving slow. It also means that things that are far away can seem to be closer than things that are nearby.

The general theory of relativity is all about gravity. It tells us that gravity is not just a force that pulls things towards each other, but it is actually a curve in space-time that is caused by the presence of mass. This means that things with mass, like planets and stars, can 